# Advent of Code #

In [2]:
# set up the environment
import numpy as np

## Day 16: Packet Decoder ##

As you leave the cave and reach open waters, you receive a transmission from the Elves back on the ship.

The transmission was sent using the Buoyancy Interchange Transmission System (BITS), a method of packing numeric expressions into a binary sequence. Your submarine's computer has saved the transmission in hexadecimal (your puzzle input).

The first step of decoding the message is to convert the hexadecimal representation into binary. Each character of hexadecimal corresponds to four bits of binary data:

0 = 0000  
1 = 0001  
2 = 0010  
3 = 0011  
4 = 0100  
5 = 0101  
6 = 0110  
7 = 0111  
8 = 1000  
9 = 1001  
A = 1010  
B = 1011  
C = 1100  
D = 1101  
E = 1110  
F = 1111  

The BITS transmission contains a single packet at its outermost layer which itself contains many other packets. The hexadecimal representation of this packet might encode a few extra 0 bits at the end; these are not part of the transmission and should be ignored.

Every packet begins with a standard header: the first three bits encode the packet version, and the next three bits encode the packet type ID. These two values are numbers; all numbers encoded in any packet are represented as binary with the most significant bit first. For example, a version encoded as the binary sequence 100 represents the number 4.

Packets with type ID 4 represent a literal value. Literal value packets encode a single binary number. To do this, the binary number is padded with leading zeroes until its length is a multiple of four bits, and then it is broken into groups of four bits. Each group is prefixed by a 1 bit except the last group, which is prefixed by a 0 bit. These groups of five bits immediately follow the packet header. For example, the hexadecimal string D2FE28 becomes:

110100101111111000101000  
VVVTTTAAAAABBBBBCCCCC

Below each bit is a label indicating its purpose:

    The three bits labeled V (110) are the packet version, 6.
    The three bits labeled T (100) are the packet type ID, 4, which means the packet is a literal value.
    The five bits labeled A (10111) start with a 1 (not the last group, keep reading) and contain the first four bits of the number, 0111.
    The five bits labeled B (11110) start with a 1 (not the last group, keep reading) and contain four more bits of the number, 1110.
    The five bits labeled C (00101) start with a 0 (last group, end of packet) and contain the last four bits of the number, 0101.
    The three unlabeled 0 bits at the end are extra due to the hexadecimal representation and should be ignored.

So, this packet represents a literal value with binary representation 011111100101, which is 2021 in decimal.

Every other type of packet (any packet with a type ID other than 4) represent an operator that performs some calculation on one or more sub-packets contained within. Right now, the specific operations aren't important; focus on parsing the hierarchy of sub-packets.

An operator packet contains one or more packets. To indicate which subsequent binary data represents its sub-packets, an operator packet can use one of two modes indicated by the bit immediately after the packet header; this is called the length type ID:

    If the length type ID is 0, then the next 15 bits are a number that represents the total length in bits of the sub-packets contained by this packet.
    If the length type ID is 1, then the next 11 bits are a number that represents the number of sub-packets immediately contained by this packet.

Finally, after the length type ID bit and the 15-bit or 11-bit field, the sub-packets appear.

For example, here is an operator packet (hexadecimal string 38006F45291200) with length type ID 0 that contains two sub-packets:

00111000000000000110111101000101001010010001001000000000  
VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB

    The three bits labeled V (001) are the packet version, 1.
    The three bits labeled T (110) are the packet type ID, 6, which means the packet is an operator.
    The bit labeled I (0) is the length type ID, which indicates that the length is a 15-bit number representing the number of bits in the sub-packets.
    The 15 bits labeled L (000000000011011) contain the length of the sub-packets in bits, 27.
    The 11 bits labeled A contain the first sub-packet, a literal value representing the number 10.
    The 16 bits labeled B contain the second sub-packet, a literal value representing the number 20.

After reading 11 and 16 bits of sub-packet data, the total length indicated in L (27) is reached, and so parsing of this packet stops.

As another example, here is an operator packet (hexadecimal string EE00D40C823060) with length type ID 1 that contains three sub-packets:

11101110000000001101010000001100100000100011000001100000  
VVVTTTILLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBCCCCCCCCCCC

    The three bits labeled V (111) are the packet version, 7.
    The three bits labeled T (011) are the packet type ID, 3, which means the packet is an operator.
    The bit labeled I (1) is the length type ID, which indicates that the length is a 11-bit number representing the number of sub-packets.
    The 11 bits labeled L (00000000011) contain the number of sub-packets, 3.
    The 11 bits labeled A contain the first sub-packet, a literal value representing the number 1.
    The 11 bits labeled B contain the second sub-packet, a literal value representing the number 2.
    The 11 bits labeled C contain the third sub-packet, a literal value representing the number 3.

After reading 3 complete sub-packets, the number of sub-packets indicated in L (3) is reached, and so parsing of this packet stops.

For now, parse the hierarchy of the packets throughout the transmission and add up all of the version numbers.

Here are a few more examples of hexadecimal-encoded transmissions:

    8A004A801A8002F478 represents an operator packet (version 4) which contains an operator packet (version 1) which contains an operator packet (version 5) which contains a literal value (version 6); this packet has a version sum of 16.
    620080001611562C8802118E34 represents an operator packet (version 3) which contains two sub-packets; each sub-packet is an operator packet that contains two literal values. This packet has a version sum of 12.
    C0015000016115A2E0802F182340 has the same structure as the previous example, but the outermost packet uses a different length type ID. This packet has a version sum of 23.
    A0016C880162017C3686B18A3D4780 is an operator packet that contains an operator packet that contains an operator packet that contains five literal values; it has a version sum of 31.

Decode the structure of your hexadecimal-encoded BITS transmission; what do you get if you add up the version numbers in all packets?

In [3]:
def parse_literal(string_to_parse: str) -> tuple[int, int]:
    """ Parse a literal value.

    Keyword arguments:
    string_to_parse -> the string to parse
    
    Returns:
        A tuple consisting of the number of items parsed and 
        the value of the parsed items
    """
    literal_value = "0b"
    x = 0
    while x < len(string_to_parse):
        if string_to_parse[x] == '1':
            literal_value += string_to_parse[x+1:x+5]
            x += 5
        else:
            literal_value += string_to_parse[x+1:x+5]
            x += 5
            break
        
    return x, int(literal_value, 2)

def parse_operator(string_to_parse):
    assert string_to_parse[0] == '0' or string_to_parse[0] == '1'
    x = 0
    sub_packet_len = 0
    if string_to_parse[0] == '0':
        sub_packet_len = int(string_to_parse[1:16],)
    else:
        pass


bitsinput: int = 0x38006F45291200
# bitsinput = 0xD2FE28
bit_string = str(bin(bitsinput))[2:]
padding_required = (4 - len(bit_string) % 4) % 4
if padding_required != 0:
    bit_string = ('0' * padding_required) + bit_string

bit_pos = 0
while bit_pos < len(bit_string):
    package_version=bit_string[bit_pos: bit_pos+3]
    bit_pos += 3
    package_type=bit_string[bit_pos: bit_pos + 3]
    bit_pos += 3
    if package_type == '100':
        bits_consumed, literal= parse_literal(bit_string[bit_pos:])
        bit_pos += bits_consumed
        print(f'Read {bits_consumed} and got {literal}')

    else:
        parse_operator(bit_string[bit_pos:])


Read 5 and got 9


IndexError: string index out of range

In [4]:
class Transmission:
    '''Transmission class holds a list of packets '''

    def __init__(self, packet_string: str) -> None:
        padding = (4 - len(packet_string) % 4) %4
        if padding:
            packet_string = ('0' * padding) + packet_string
        self.version = int(packet_string[:3], 2)
        self.type = int(packet_string[3:6], 2)
        self.data = packet_string[6:]

    def __str__(self) -> str:
        return f"version: {self.version}\ntype: {self.type}\ndata: {self.data}"

    def parse():
        pass

class Packet:
    '''Class Packet is a base class for packet data'''

    def __init__(self, packet_string) -> None:
        self.packet_version = int(packet_string[:3], 2)
        self.packet_type = int(packet_string[3:6], 2)
        self.packet_data = packet_string[6:]
        self.packet_len = len(self.data)

    def version(self) -> int:
        return self.packet_version
    
    def type(self) -> int:
        return self.packet_type

    def length(self)-> int:
        return self.packet_len

    def data(self) -> str:
        return self.packet_data

class LiteralPacket(Packet):
    '''
    Class packet extracts the literal part of the input string and discards the rest
    '''

    def __init__(self, packet_string) -> None:
        super().__init__(packet_string)
        self.literal_str = "0b"
        x = 0
        while x < len(self.packet_data):
            if self.pa[x] == '1':
                self.literal_str += self.packet_data[x+1:x+5]
                x += 5
            else:
                self.literal_str += self.packet_data[x+1:x+5]
                x += 5
                break

        self.packet_len = x

    def value(self)-> int:
        return int(self.literal_str, 2)


class OperatorPacket(Packet):
    pass


trans_input = 0xD2FE28
trans_string = str(bin(trans_input))[2:]
first_packet = Transmission(trans_string)

trans_input = 0x38006F45291200
trans_string = str(bin(trans_input))[2:]
second_packet = Transmission(trans_string)
print(first_packet)
print(second_packet)




version: 6
type: 4
data: 101111111000101000
version: 1
type: 6
data: 00000000000110111101000101001010010001001000000000
